In [1]:
%matplotlib notebook

In [2]:
import os
import shutil
import sys
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Use OpenStudio to create a Model

In [3]:
import openstudio

In [4]:
# For reproducibility, here are the versions I used to create and run this notebook
_s = !pip list
print(f"Pip package used initially: {[x for x in _s if 'openstudio' in x][0]}")
print(f"OpenStudio Long Version:    {openstudio.openStudioLongVersion()}")

Pip package used initially: openstudio                        3.3.0
OpenStudio Long Version:    3.3.0+b79cf6780c


## Loading the model

In [5]:
osmpath = openstudio.toPath('As-Built-input.osm')

m = openstudio.model.Model.load(osmpath).get()
# print(m)

## Find the Hot Water Plant Loop

In [6]:
plantLoop = m.getPlantLoops()
for loop in plantLoop:
    if 'Hot Water Loop' in loop.name().get():
        print(loop)
        hwLoop = loop

OS:PlantLoop,
  {cdc8bbdb-db5a-4a1c-a7ec-4fec1f98c377}, !- Handle
  Hot Water Loop,                         !- Name
  Water,                                  !- Fluid Type
  0,                                      !- Glycol Concentration
  ,                                       !- User Defined Fluid Type
  ,                                       !- Plant Equipment Operation Heating Load
  ,                                       !- Plant Equipment Operation Cooling Load
  ,                                       !- Primary Plant Equipment Operation Scheme
  {b99e7591-a8dd-4ad2-af2b-800bacd73127}, !- Loop Temperature Setpoint Node Name
  ,                                       !- Maximum Loop Temperature {C}
  ,                                       !- Minimum Loop Temperature {C}
  ,                                       !- Maximum Loop Flow Rate {m3/s}
  ,                                       !- Minimum Loop Flow Rate {m3/s}
  Autocalculate,                          !- Plant Loop Volu

## Filter Reheat Coils and Connect to Hot Water Loop

Find all the coils with no Water Inlet Node Name and connect to the plant loop called Hot Water Loop

In [7]:
reheat = m.getCoilHeatingWaters()
for coil in reheat:
    # print(coil)
    if len(coil.getString(5).get()) < 5:
        hwLoop.addDemandBranchForComponent(coil)
        print(coil)

OS:Coil:Heating:Water,
  {f6e18e64-db8e-4779-9dd7-f66be627a23b}, !- Handle
  Coil Heating Water 138,                 !- Name
  {fb414a7f-6f72-4969-9d94-727b357477b5}, !- Availability Schedule Name
  ,                                       !- U-Factor Times Area Value {W/K}
  ,                                       !- Maximum Water Flow Rate {m3/s}
  {9c5c5516-382e-43ea-b504-51b9cdff9234}, !- Water Inlet Node Name
  {b49e0400-9298-41b6-9d32-944245859b7d}, !- Water Outlet Node Name
  ,                                       !- Air Inlet Node Name
  ,                                       !- Air Outlet Node Name
  ,                                       !- Performance Input Method
  ,                                       !- Rated Capacity {W}
  ,                                       !- Rated Inlet Water Temperature {C}
  ,                                       !- Rated Inlet Air Temperature {C}
  ,                                       !- Rated Outlet Water Temperature {C}
  ,           

# Save the model

In [8]:
openstudio.model.Model.save(m, "As-Built.osm", True)

True